In [ ]:
!pip install torch sklearn

  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [ ]:
from google.colab import files

# Upload intents.json
uploaded = files.upload()

# Upload knowledge_base.json (if needed)
uploaded = files.upload()


Saving intents.json to intents.json


Saving knowledge_base.json to knowledge_base.json


In [ ]:
import json

# Load the intents file
with open('intents.json', 'r') as f:
    intents = json.load(f)

# Count the number of unique tags
tags = [intent['tag'] for intent in intents['intents']]
unique_tags = set(tags)
print(f"Number of unique tags: {len(unique_tags)}")


Number of unique tags: 51


In [1]:
import json
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from torch.utils.data import Dataset, DataLoader
import pickle

# Load intents file
with open('intents.json', 'r') as f:
    intents = json.load(f)

# Prepare data
patterns = []
tags = []

for intent in intents['intents']:
    for pattern in intent['patterns']:
        patterns.append(pattern)
        tags.append(intent['tag'])

# Convert labels to integers
label_encoder = LabelEncoder()
encoded_tags = label_encoder.fit_transform(tags)

# Tokenization function
def tokenize(text):
    return text.split()

# Convert text to vectors
vectorizer = CountVectorizer(tokenizer=tokenize, binary=True)
X = vectorizer.fit_transform(patterns).toarray()
y = encoded_tags

# Create custom dataset
class IntentDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return torch.tensor(self.X[idx], dtype=torch.float32), torch.tensor(self.y[idx], dtype=torch.long)

dataset = IntentDataset(X, y)
dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

# Define the model
class IntentClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(IntentClassifier, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

input_dim = X.shape[1]
hidden_dim = 128
output_dim = len(set(tags))

model = IntentClassifier(input_dim, hidden_dim, output_dim)

# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
epochs = 100
for epoch in range(epochs):
    total_loss = 0
    for inputs, labels in dataloader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    if (epoch+1) % 10 == 0:
        print(f'Epoch {epoch+1}/{epochs}, Loss: {total_loss / len(dataloader)}')

# Save the model
model_save_path = 'intent_classifier.pth'
torch.save(model.state_dict(), model_save_path)
print(f'Model saved as {model_save_path}')

# Save label encoder and vectorizer for future use
with open('label_encoder.pkl', 'wb') as f:
    pickle.dump(label_encoder, f)
with open('vectorizer.pkl', 'wb') as f:
    pickle.dump(vectorizer, f)
print('Label encoder and vectorizer saved.')

# Function to predict intent
def predict_intent(text, model, vectorizer, label_encoder):
    text_vector = vectorizer.transform([text]).toarray()
    text_tensor = torch.tensor(text_vector, dtype=torch.float32)
    output = model(text_tensor)
    _, predicted = torch.max(output, dim=1)
    intent = label_encoder.inverse_transform(predicted.numpy())
    return intent[0]

# Load the model for inference
model.load_state_dict(torch.load('intent_classifier.pth'))
model.eval()

# Example query
query = "Tell me about NZXT H5 Flow"
predicted_intent = predict_intent(query, model, vectorizer, label_encoder)
print(f'Predicted Intent: {predicted_intent}')


/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Epoch 10/100, Loss: 2.4130412502722307
Epoch 20/100, Loss: 0.6322479139674794
Epoch 30/100, Loss: 0.20419883626428517
Epoch 40/100, Loss: 0.10671159506521442
Epoch 50/100, Loss: 0.07087105114690283
Epoch 60/100, Loss: 0.0563456808978861
Epoch 70/100, Loss: 0.04438889058391479
Epoch 80/100, Loss: 0.040504034058275545
Epoch 90/100, Loss: 0.03660241000100293
Epoch 100/100, Loss: 0.03423254801468416
Model saved as intent_classifier.pth
Label encoder and vectorizer saved.
Predicted Intent: product_query
